In [ ]:
import sys; sys.path.append('..')

In [ ]:
from itertools import combinations
import pickle
from collections import defaultdict
from fractions import Fraction
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from util import step_plot, ds_names, ds_names_short, colors_full, linestyles, colors_group
import networkx as nx
from scipy.stats import pearsonr
import seaborn as sns
from collections import deque

In [ ]:
p_CE = Path('../data/clique_expansion')
p_LCC = Path('../data/largest_connected_component')
p_LCC.mkdir(exist_ok=True)
for ds_name in ds_names:
    p_CE_ds = p_CE / ds_name
    with (p_CE_ds / 'CE_U.pkl').open('rb') as f:
        CE_U = pickle.load(f)
    LCC = max(nx.connected_components(CE_U), key=len)
    with (p_LCC / '{}.pkl'.format(ds_name)).open('wb') as f:
        pickle.dump(LCC, f)
    print('Dataset {},'.format(ds_name), 'total number of nodes = {},'.format(CE_U.number_of_nodes()),
          'number of nodes in the LCC = {}'.format(len(LCC)))

In [ ]:
p_data_cpp = Path('../data_cpp')
p_data_cpp.mkdir(exist_ok=True)
p_incident = Path('../data/incident')
p_LCC = Path('../data/largest_connected_component')

for ds_name in ds_names:
    print(ds_name)
    p_incident_ds = p_incident / ds_name
    with (p_incident_ds / 'i2edges.pkl').open('rb') as f:
        i2edges = pickle.load(f)
    with (p_incident_ds / 'v2edges.pkl').open('rb') as f:
        v2edges = pickle.load(f)

    with open(p_data_cpp / '{}_edges.txt'.format(ds_name), 'w') as f:
        for e in i2edges.values():
            f.write('\t'.join([str(v) for v in e]) + '\n')

    with open(p_data_cpp / '{}_degrees.txt'.format(ds_name), 'w') as f:
        for v, E_v in v2edges.items():
            f.write('{}\t{}\n'.format(v, len(E_v)))

    with open(p_data_cpp / '{}_incident.txt'.format(ds_name), 'w') as f:
        for v, E_v in v2edges.items():
            f.write(str(v) + '\t' + ','.join([str(e) for e in E_v]) + '\n')

    with (p_LCC / '{}.pkl'.format(ds_name)).open('rb') as f:
        LCC = pickle.load(f)

    with open(p_data_cpp / '{}_cc.txt'.format(ds_name), 'w') as f:
        for v in LCC:
            f.write('{}\n'.format(v))
